In [22]:
import numpy as np
import cv2


You are given three cameras (1, 2 and 3) that share the same camera matrix K and have the following extrinsics. 
You observe the same point in all three cameras, but with some noise. The
observed points for cameras 1 to 3 are respectively:

In [23]:
K = np.array([[900, 0, 1070], [0, 900, 610.0], [0, 0, 1]], float)
R1 = cv2.Rodrigues(np.array([-1.6, 0.3, -2.1]))[0]
t1 = np.array([[0.0], [1.0], [3.0]], float)
R2 = cv2.Rodrigues(np.array([-0.4, -1.3, -1.6]))[0]
t2 = np.array([[0.0], [1.0], [6.0]], float)
R3 = cv2.Rodrigues(np.array([2.5, 1.7, -0.4]))[0]
t3 = np.array([[2.0], [-7.0], [25.0]], float)

In [24]:
p1 = np.array([[1046.0], [453.0]])
p2 = np.array([[1126.0], [671.0]])
p3 = np.array([[1165.0], [453.0]])

How far is p2 from the epipolar line in camera 2 corresponding to p1?

In [25]:
import sys
import os

# Get the absolute path to the parent directory containing "02504 Computer Vision"
parent_dir = os.path.abspath(os.path.join("../../..", "02504-Computer-Vision"))

# Add it to sys.path
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from utility import Pi, Piinv, skew, projectpoints, triangulate

![relative](images/relative.png)

In [26]:
rot = R2 @ R1.T
t = t2 - rot @ t1
rot, t

(array([[-0.00896299, -0.53095939, -0.84734986],
        [ 0.94414643,  0.27464955, -0.18208552],
        [ 0.32940427, -0.80165438,  0.49884174]]),
 array([[3.07300898],
        [1.271607  ],
        [5.30512918]]))

In [27]:
E = skew(t) @ rot
F = np.linalg.inv(K.T) @ E @ np.linalg.inv(K)
F

array([[-5.66660003e-06, -3.05733414e-06,  9.70636677e-03],
       [-1.30841003e-06, -4.36193890e-07, -5.03197367e-03],
       [ 1.00978009e-02,  5.22539492e-03, -1.12909814e+01]])

In [28]:
l = F @ Piinv(p1)
l

array([[ 0.00239413],
       [-0.00659817],
       [ 1.6384222 ]])

In [29]:
q = Piinv(p2)
d = np.linalg.norm(l.T @ q) / (
    np.linalg.norm(q[-1]) * np.sqrt(l[0] ** 2.0 + l[1] ** 2.0)
)
d

array([13.27182907])

## Alternative code

In [30]:
import numpy as np
import cv2

# Camera intrinsic matrix
K = np.array([[900, 0, 1070], [0, 900, 610.0], [0, 0, 1]], float)

# Camera 1 pose
R1 = cv2.Rodrigues(np.array([-1.6, 0.3, -2.1]))[0]
t1 = np.array([[0.0], [1.0], [3.0]], float)

# Camera 2 pose
R2 = cv2.Rodrigues(np.array([-0.4, -1.3, -1.6]))[0]
t2 = np.array([[0.0], [1.0], [6.0]], float)

# Points
p1_homo = np.array([[1046.0], [453.0], [1.0]])  # Homogeneous
p2_homo = np.array([[1126.0], [671.0], [1.0]])  # Homogeneous

# Compute relative pose
R1_inv = R1.T  # Inverse of rotation matrix is its transpose
R_2to1 = R2 @ R1_inv
t_2to1 = t2 - R2 @ R1_inv @ t1


# Skew-symmetric matrix of t_2to1
def skew_symmetric(t):
    return np.array([[0, -t[2], t[1]], [t[2], 0, -t[0]], [-t[1], t[0], 0]], float)


t_skew = skew_symmetric(t_2to1.flatten())

# Compute K^-1 and K^-T
K_inv = np.linalg.inv(K)
K_inv_T = K_inv.T

# Fundamental matrix
F = K_inv_T @ t_skew @ R_2to1 @ K_inv

# Epipolar line in camera 2
l2 = F @ p1_homo

# Distance from p2 to the epipolar line
a, b, c = l2.flatten()
distance = abs(a * 1126 + b * 671 + c) / np.sqrt(a**2 + b**2)

print(f"Distance from p2 to the epipolar line: {distance:.6f} pixels")

Distance from p2 to the epipolar line: 13.271829 pixels


Use all three observations of the point from the previous question to triangulate the point with the linear algorithm from the slides. Do not normalize the points.

What is the triangulated point?

In [31]:
p1.shape, p2.shape, p3.shape

((2, 1), (2, 1), (2, 1))

In [32]:
P1 = K @ np.concatenate((R1, t1), axis=1)
P2 = K @ np.concatenate((R2, t2), axis=1)
P3 = K @ np.concatenate((R3, t3), axis=1)
project_matrices = np.array([P1, P2, P3])
pixel_coords = np.array([p1, p2, p3])
project_matrices.shape, pixel_coords.shape

((3, 3, 4), (3, 2, 1))

In [33]:
triangulate(pixel_coords, project_matrices)

Vt is [[-0.29966278 -0.1237932   0.21481617  0.92126624]
 [ 0.2558236  -0.94558195 -0.20104693  0.00303097]
 [-0.01146078 -0.20499066  0.94575103 -0.25179847]
 [-0.9190349  -0.22029263 -0.13780109 -0.29640659]]


array([3.10058867, 0.74321098, 0.46490561, 1.        ])

## Alternative
cv2.triangulatePoints does not accept projPoints3 or projMatr3 as arguments. It only works with two views.

In [ ]:
# points_2d = np.hstack((p1, p2, p3))
# points = cv2.triangulatePoints(projPoints1=p1, projPoints2=p2, projPoints3=p3, projMatr1=P1, projMatr2=P2, projMatr3=P3)

error: OpenCV(4.11.0) :-1: error: (-5:Bad argument) in function 'triangulatePoints'
> Overload resolution failed:
>  - triangulatePoints() takes at most 5 keyword arguments (6 given)
>  - triangulatePoints() takes at most 5 keyword arguments (6 given)
